# Who is the next Sunil Chhetri?

We will use the data from the past two ISL seasons and compare the attributes of Chhetri with other strikers in Indian Football to see who is capable of replacing him.

## Read and explore the data

In [ ]:
# Import required libraries:
import io
import numpy as np
import pandas as pd
from copy import deepcopy
from google.colab import files

In [ ]:
# Import plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
uploaded = files.upload()

In [ ]:
isl_df = pd.read_csv(io.BytesIO(uploaded['isl_player_final.csv']))
isl_df

In [ ]:
isl_df.info()

In [ ]:
isl_df.describe().round(1)

In [ ]:
isl_df[(isl_df["minutes_played"] > 1500) & (isl_df["country_id"] == 1)].shape

In [ ]:
isl_df[(isl_df["country_id"] == 1) & (isl_df["position_id"] == 2)]

In [ ]:
indian_forwards = deepcopy(isl_df[(isl_df["country_id"] == 1) & (isl_df["position_id"] == 2)])

In [ ]:
indian_forwards.reset_index(drop=True, inplace=True)

In [ ]:
indian_forwards[indian_forwards["name"].str.contains("Singh")]

## Level the playing field

Why do we need this?

In [ ]:
fwd_mins_df = indian_forwards.groupby(["id", "name"])["minutes_played"].sum().reset_index()

In [ ]:
px.bar(x="name", y="minutes_played", text="minutes_played", data_frame=fwd_mins_df)

In [ ]:
indian_forwards["goalsPer90"] = indian_forwards["events.goals"].divide(indian_forwards["minutes_played"]).multiply(90)

In [ ]:
indian_forwards = indian_forwards[indian_forwards["minutes_played"] > 0]

In [ ]:
indian_forwards["assistsPer90"] = indian_forwards["events.assists"].divide(indian_forwards["minutes_played"]).multiply(90)
indian_forwards["shots_on_targetPer90"] = indian_forwards["events.shots_on_target"].divide(indian_forwards["minutes_played"]).multiply(90)

In [ ]:
per90Cols = ["events.goals", "events.assists", "events.shots", "events.shots_on_target", "events.chances_created",
             "events.fouls_suffered", "touches.total", 'touches.aerial_duel.won']
for col in per90Cols:
    indian_forwards[col + "Per90"] = indian_forwards[col].divide(indian_forwards["minutes_played"]).multiply(90)

In [ ]:
cols_for_radar = []
for i in per90Cols:
    cols_for_radar.append(i + "Per90")
print(cols_for_radar)

In [ ]:
# Use list comprehension
[i + "Per90" for i in per90Cols]

In [ ]:
cols_for_radar = [i + "Per90" for i in per90Cols]
print(cols_for_radar)

In [ ]:
indian_forwards.loc[(indian_forwards["id"] == 19150), cols_for_radar].sum()

## Chhetri's Performance

Let's use some key metrics to assess Chhetri's performance

### What is a Radar Plot?

r --> All the values in the Radar Plot

theta --> All the labels in the Radar Plot

In [ ]:
df = pd.DataFrame(dict(
    r=[1, 5, 2, 2, 3],
    theta=['processing cost','mechanical properties','chemical stability',
           'thermal stability', 'device integration']))
fig = px.line_polar(df, r='r', theta='theta', line_close=True)
fig.show()

### Radar Plot for Chhetri's performance

In [ ]:
indian_forwards.loc[(indian_forwards["id"] == 19150), cols_for_radar].sum()

In [ ]:
fig = px.line_polar(indian_forwards, r=indian_forwards.loc[(indian_forwards["id"] == 19150), cols_for_radar].sum(),
                    theta=cols_for_radar, line_close=True)
fig.update_traces(fill='toself')
fig.show()

### Normalization of Values

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
indian_forwards[cols_for_radar] = scaler.fit_transform(indian_forwards[cols_for_radar])

In [ ]:
indian_forwards[cols_for_radar].max()

### Normalized Radar Plot for Chhetri's performance

In [ ]:
fig = px.line_polar(indian_forwards, r=indian_forwards.loc[(indian_forwards["id"] == 19150), cols_for_radar].sum(),
                    theta=cols_for_radar, line_close=True)
fig.update_traces(fill='toself')
fig.show()

## The Others

Here we try to have a look at other players and compare them with Chhetri

In [ ]:
indian_forwards[indian_forwards["name"].str.contains("Manvir")]

In [ ]:
df

In [ ]:
for i, row in indian_forwards.iterrows():
    print("Index location -->", i, "\n")
    print(row.values, "\n")

In [ ]:
fig = px.line_polar(indian_forwards, r=indian_forwards.loc[(indian_forwards["id"] == 19150), cols_for_radar].sum(),
                    theta=cols_for_radar, line_close=True)
fig.update_traces(fill='toself')
fig.show()

In [ ]:
for i, row in indian_forwards.iterrows():
    print(row["name"])
    fig = px.line_polar(indian_forwards, r=indian_forwards.loc[(indian_forwards["id"] == row["id"]), cols_for_radar].sum(),
                        theta=cols_for_radar, line_close=True,
                        title=row["name"])
    fig.update_traces(fill='toself')
    fig.show()

## The Replacement

We will use something called as "Multiple Trace Radar Chart" to perform comparative analysis.

In [ ]:
ind_fwds_id_names = indian_forwards.drop_duplicates(subset=["id"])[["id", "name"]]

In [ ]:
ind_fwds_id_names.shape

In [ ]:
ind_fwds_id_names.head()

In [ ]:
indian_forwards[cols_for_radar].max().max()

In [ ]:
isl_max = indian_forwards[cols_for_radar].max().max()

In [ ]:
import plotly.graph_objects as go

In [ ]:
for i, row in ind_fwds_id_names.iterrows():
    if row["id"] == 19150:
        continue
    print(row["name"])
    # Initiate the plotly go figure
    fig = go.Figure()
    # Add Radar plots for different players:
    fig.add_trace(go.Scatterpolar(
        r=indian_forwards.loc[(indian_forwards["id"] == 19150), cols_for_radar].sum(),
        theta=cols_for_radar,
        fill='toself',
        name='Sunil Chhetri'))
    fig.add_trace(go.Scatterpolar(
        r=indian_forwards.loc[(indian_forwards["id"] == row["id"]), cols_for_radar].sum(),
        theta=cols_for_radar,
        fill='toself',
        name=row["name"]))
    # Additional properties for the plot:
    fig.update_layout(
        title="Sunil Chhetri vs " + row["name"],
    polar=dict(
        radialaxis=dict(
        visible=True,
        range=[0, isl_max]
        )),
    showlegend=True
    )
    fig.show()